In [3]:
# Imports
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os

import sys
sys.path.append('../../../')

from src import customFunc as cf
from src import ansatzs as anz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.providers.fake_provider import FakeManila
import time

import pyscf
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.providers.fake_provider import FakeGuadalupeV2
from qiskit.providers.aer import AerSimulator

## H8

In [16]:
# Cargamos los archivos

hamiltonians = {}
nuclear_repulsions = {}
ansatzes = {}

dist = 0.700

for n_atoms in [2, 4, 6, 8]:
    molecule_id = f"H{n_atoms}"

    with open(f"data/hamiltonian_{molecule_id}_{dist:.3f}.pkl", "rb") as f:
        hamiltonians[molecule_id] = pickle.load(f)

    with open(f"data/nuclear_repulsion_{molecule_id}_{dist:.3f}.pkl", "rb") as f:
        nuclear_repulsions[molecule_id] = pickle.load(f)

    with open(f"data/ansatz_{molecule_id}_{dist:.3f}.pkl", "rb") as f:
        ansatzes[molecule_id] = pickle.load(f)

In [ ]:
ansatz_naive = ansatzes['H8']
hamiltonan = hamiltonians['H8']

backend = FakePrague()

# Transpile circuit with a pass manager for a backend
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
t0 = time.time()
transpiled_ansatz_naive = pm.run(ansatz_naive)
t1 = time.time()
t_optimized=t1 - t0

# Show results
print(f"Transpilation time: {t_optimized} (s)")
print(f'Circuit info: {transpiled_ansatz_naive.count_ops()}')
print(f'Depth: {transpiled_ansatz_naive.depth()}')

# ansatz_naive.decompose().draw('mpl')

Transpilation time: 218.13428950309753 (s)
Circuit info: OrderedDict([('cx', 43715), ('rz', 28329), ('sx', 18722), ('x', 1825)])
Depth: 66048


In [19]:
ansatz_opt=anz.optimize_ansatz(ansatz_naive)

# Transpile circuit with a pass manager for a backend
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
t0 = time.time()
transpiled_ansatz_opt = pm.run(ansatz_opt)
t1 = time.time()
t_optimized=t1 - t0

# Show results
print(f"Transpilation time: {t_optimized} (s)")
print(f'Circuit info: {transpiled_ansatz_opt.count_ops()}')
print(f'Depth: {transpiled_ansatz_opt.depth()}')

Transpilation time: 134.2523169517517 (s)
Circuit info: OrderedDict([('cx', 44700), ('rz', 25178), ('sx', 16235), ('x', 1077)])
Depth: 63967


## O3

In [ ]:
# Esto es para generar el ansatz del O3 (No hace falta ejecutar)

# Parámetros
dist = 1.28  # distancia O-O en Ångstroms
angle = 117 * np.pi / 180  # ángulo en radianes

# Coordenadas
x1, y1 = -dist/2 * np.cos(angle/2), dist/2 * np.sin(angle/2)
x3, y3 = dist/2 * np.cos(angle/2), dist/2 * np.sin(angle/2)

geometry = f"O {x1:.4f} {y1:.4f} 0.0; O 0.0 0.0 0.0; O {x3:.4f} {y3:.4f} 0.0"

# Configuramos el driver PySCF
driver = PySCFDriver(atom=geometry, basis='sto3g')

# Ejecutamos el driver para obtener el problema de la estructura electrónica
es_problem = driver.run()

# Construimos el Hamiltoniano después de la segunda cuantización
hamiltonian = es_problem.second_q_ops()[0]

# Aplicamos las transformaciones de Jordan-Wigner
mapper = JordanWignerMapper()
hamiltonian = mapper.map(hamiltonian)

# Ansatz
init_state = HartreeFock(
    num_spatial_orbitals=es_problem.num_spatial_orbitals,
    num_particles=es_problem.num_particles,
    qubit_mapper=mapper
)

ansatz = UCCSD(
    num_spatial_orbitals=es_problem.num_spatial_orbitals,
    num_particles=es_problem.num_particles,
    qubit_mapper=mapper,
    initial_state=init_state
)

base_path = f"notebooks/VQE/H_strings (UCCSD)/data/"
with open(f"{base_path}ansatz_O3.pkl", "wb") as f:
        pickle.dump(ansatz, f)

In [27]:
with open(f"data/ansatz_O3.pkl", "rb") as f:
    ansatz_naive = pickle.load(f)

backend = AerSimulator()

# Transpile circuit with a pass manager for a backend
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
t0 = time.time()
transpiled_ansatz_naive = pm.run(ansatz_naive)
t1 = time.time()
t_optimized=t1 - t0

# Show results
print(f"Transpilation time: {t_optimized} (s)")
print(f'Circuit info: {transpiled_ansatz_naive.count_ops()}')
print(f'Depth: {transpiled_ansatz_naive.depth()}')

# ansatz_naive.decompose().draw('mpl')

Transpilation time: 1475.3436632156372 (s)
Circuit info: OrderedDict([('cx', 328368), ('u2', 31349), ('rz', 13680), ('unitary', 4050), ('u3', 22)])
Depth: 354614


In [28]:
ansatz_opt=anz.optimize_ansatz(ansatz_naive)

# Transpile circuit with a pass manager for a backend
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
t0 = time.time()
transpiled_ansatz_opt = pm.run(ansatz_opt)
t1 = time.time()
t_optimized=t1 - t0

# Show results
print(f"Transpilation time: {t_optimized} (s)")
print(f'Circuit info: {transpiled_ansatz_opt.count_ops()}')
print(f'Depth: {transpiled_ansatz_opt.depth()}')

Transpilation time: 1141.578495502472 (s)
Circuit info: OrderedDict([('cx', 328368), ('u2', 31349), ('rz', 13680), ('unitary', 4050), ('u3', 22)])
Depth: 354614
